# Hello World

In [2]:

from PIL import Image
from pytesseract import image_to_string
import io
from docx import Document
import re
import pandas as pd
import cv2
import numpy as np
from bs4 import BeautifulSoup
import requests

In [3]:
df = pd.read_excel("anskey.xlsx")
ans_key = df.set_index('Question ID')['Correct Option ID'].to_dict()

In [4]:
df_cuet = pd.read_excel("CUETFinal.xlsx")


In [34]:
df_cuet

,URL,Name,Subject,Type
0,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Accountancy 05/25 1Morning,Accountancy,Image
1,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Accountancy 05/26 1Morning,Accountancy,Image
2,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Accountancy 05/27 1Morning,Accountancy,Image
3,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Accountancy 05/28 3Evening,Accountancy,Image
4,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Accountancy 05/29 3Evening,Accountancy,Image
...,...,...,...,...
734,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Telugu 05/23 1Morning,Telugu,Image
735,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Urdu 05/23 1Morning,Urdu,Image
736,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Urdu 05/24 1Morning,Urdu,Image
737,https://cdn3.digialm.com//per/g28/pub/2083/tou...,Urdu 06/06 2Afternoon,Urdu,Image


In [16]:
doc = Document()
doc.add_heading('CUET Questions', 0)

In [41]:
url = 'https://cdn3.digialm.com//per/g28/pub/2083/touchstone/AssessmentQPHTMLMode1//2083O23168/2083O23168S29D578/16866457938406188/PB11375067_2083O23168S29D578E1.html'
page = requests.get(url)
page_content = BeautifulSoup(page.content, "lxml")
url

'https://cdn3.digialm.com//per/g28/pub/2083/touchstone/AssessmentQPHTMLMode1//2083O23168/2083O23168S29D578/16866457938406188/PB11375067_2083O23168S29D578E1.html'

In [42]:

sections = page_content.find_all('div',class_="section-cntnr")
sections

[<div class="section-cntnr"> <div class="section-lbl"><span class="">Section : </span><span class="bold">Compulsory</span></div> <div class="question-pnl" style="page-break-inside: avoid;"> <table border="0" cellpadding="0" cellspacing="0" class="questionPnlTbl" width="100%"><tr><td class="rw"><table border="0" cellpadding="0" cellspacing="0" class="questionRowTbl" style="float: left;" width="70%"><tr></tr><tr><td align="center" class="bold" valign="top" width="7%">Q.1</td><td class="bold" style="text-align: left;" valign="top"><img name="2083_212807_0_24322650_com_set05_eng_006.jpg" src="/per/g28/pub/2083/touchstone/TempQPImagesStoreMode1/adcimages/1686645695840/21280770///2083_212807_0_24322650_com_set05_eng_006.jpg"/></td><td align="right" rowspan="5" valign="top"></td></tr><tr></tr><tr><td align="center" class="bold" style="word-break: unset;">Options</td><td style="text-align: left;">1. 1</td></tr><tr><td></td><td style="text-align: left;">2. 2</td></tr><tr><td></td><td style="tex

In [43]:
for section in sections:
    section_name = re.search(r'Section\s*:\s*(\w+\s?\w*)', section.text, re.IGNORECASE).group(1)
    print(section_name)
    questions = section.find_all('table',class_="questionPnlTbl")
    

Compulsory 
Computer Science
Informatics Practices
Physics 


In [60]:
def img_to_str(img):
    img_pil = Image.open(io.BytesIO(img))
    img = np.array(img_pil)
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    cv2.threshold(cv2.GaussianBlur(img, (5, 5), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cv2.threshold(cv2.bilateralFilter(img, 5, 75, 75), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cv2.adaptiveThreshold(cv2.GaussianBlur(img, (5, 5), 0), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    cv2.adaptiveThreshold(cv2.bilateralFilter(img, 9, 75, 75), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    text = image_to_string((img),config='--oem 3 --psm 6')
    
    processed_text = re.sub(r'(?<![^\n](\d\.|[A-Z]\.)\s)\n(?!(\d\.|[A-Z]\.)\s)', ' ', text)
    return processed_text

In [77]:
for question in questions:

    pattern = r'Question ID :(\d+)'
    match = re.search(pattern, question.text)
    
    question_id = match.group(1)
    correct_option = 0
    
    for i in range(1,5):
        option_pattern = r'Option '+str(i)+' ID :(\d+)'
        option_id = re.search(option_pattern, question.text).group(1)
        
        correct_option_id = ans_key[int(question_id)]
        
        if int(correct_option_id) == int(option_id.strip()):
            correct_option = i
            break


    question_number = re.findall(r'Q.\d+', question.text)[0]
    imgs = question.find_all("img")
    paragraph = doc.add_paragraph(str(question_number)+") ")
    prefix = "https://cdn3.digialm.com/"
    
    for im in imgs:
        
        image = requests.get(prefix+im['src'])
        paragraph.add_run(img_to_str(image.content))
    paragraph.add_run("\nCorrect Option : "+ str(correct_option)+"\n")

In [78]:
doc.save('my_document8.docx')
